In [ ]:
import warnings
import numpy as np
import pandas as pd
import plotly.express as px
from tqdm import tqdm
import util
import datasets2
from GradualSelfTrain import *

In [ ]:
settings = {
            'portraits': datasets2.load_Portraits,
            'mnist': datasets2.load_RotatedMNIST2,
           }

# neighbors = [5, 15, 30]
# metrics = ['euclidean', 'correlation']
components = np.arange(2, 15)
rep = 5

warnings.simplefilter('ignore')

for name in settings.keys():
    print(name)

    results = {}
    func = settings[name]
    x_all, y_all = func()
    for c in components:
        z_all, _ = util.fit_umap(x_all, y_all, n_components=c)
        accuracy = []
        for r in range(rep):
            clf = MLP(num_labels=np.unique(y_all[0]).size, input_dim=int(c), hidden_dim=32)
            clf, _ = train_classifier(clf, z_all[0], y_all[0])
            acc = calc_accuracy(clf, z_all[0], y_all[0])
            accuracy.append(acc)

        results[c] = (z_all, y_all, accuracy)
        pd.to_pickle(results, f'umap_{name}.pkl')